## Whitehat Hack Funds Disbursement
We need to disburse funds to users whose funds we rescued. The purpose of this notebook is to have a single source of truth for how many funds we owe each user.

#### Decisions / Assumptions
* The point in time we will use to calculate a users assets in the protocol will be the block before the first whitehat on each chain


In [2]:
import os
from web3 import Web3
from dotenv import load_dotenv

In [ ]:
####################################################
# Environment variables
####################################################
load_dotenv()
ALCHEMY_API_KEY = os.getenv('ALCHEMY_API_KEY')

####################################################
# RPC Clients
####################################################
w3_mainnet = Web3(Web3.HTTPProvider(f'https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}'))
w3_base = Web3(Web3.HTTPProvider(f'https://base-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}'))
w3_unichain = Web3(Web3.HTTPProvider(f'https://unichain-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}'))

####################################################
# Whitehat block nums
####################################################
# unichain_first_whitehat_block = 25669817
# base_first_whitehat_block = 34814419
mainnet_first_whitehat_block = 23242436

####################################################
# Subgraph urls
####################################################
chainToSubgraphUrl = {
    # mainnet
    1: 'https://api.goldsky.com/api/public/project_cl9gc21q105380hxuh8ks53k3/subgraphs/panoptic-subgraph-mainnet/dev/gn',
    # unichain
    130: 'https://api.goldsky.com/api/public/project_cl9gc21q105380hxuh8ks53k3/subgraphs/panoptic-subgraph-unichain/dev/gn',
    # base
    8453: 'https://api.goldsky.com/api/public/project_cl9gc21q105380hxuh8ks53k3/subgraphs/panoptic-subgraph-base/dev/gn',
}

####################################################
# Token addrs
####################################################
mainnet_weth = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
mainnet_wbtc = '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'
mainnet_usdc = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
mainnet_tbtc = '0x18084fbA666a33d37592fA2633fD49a74DD93a88'

####################################################
# Abis
####################################################
# Minimal ABI for balanceOf
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function"
    }
]

####################################################
# Whitehat metadata
####################################################
attacker = '0xa1F0A9d51b592ee074eD6987006976908631503B'


panoptic_safe_address = '0x82BF455e9ebd6a541EF10b683dE1edCaf05cE7A1'

## Mainnet

#### Fetch funds to disburse from Mainnet safe (as of Sep 2)

In [4]:
# Can confirm funds match safe mainnet balances here:
# https://app.safe.global/home?safe=eth:0x82BF455e9ebd6a541EF10b683dE1edCaf05cE7A1
mainnet_tue_sep_1_2025_block = 23275660

mainnet_weth_contract = w3_mainnet.eth.contract(address=mainnet_weth, abi=ERC20_ABI)
weth_balance = mainnet_weth_contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=mainnet_tue_sep_1_2025_block)

mainnet_wbtc_contract = w3_mainnet.eth.contract(address=mainnet_wbtc, abi=ERC20_ABI)
wbtc_balance = mainnet_wbtc_contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=mainnet_tue_sep_1_2025_block)

mainnet_tbtc_contract = w3_mainnet.eth.contract(address=mainnet_tbtc, abi=ERC20_ABI)
tbtc_balance = mainnet_tbtc_contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=mainnet_tue_sep_1_2025_block)

mainnet_usdc_contract = w3_mainnet.eth.contract(address=mainnet_usdc, abi=ERC20_ABI)
usdc_balance = mainnet_usdc_contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=mainnet_tue_sep_1_2025_block)

print("Mainnet funds to distribute")
print("WETH:", weth_balance)
print("WBTC:", wbtc_balance)
print("TBTC:", tbtc_balance)
print("USDC:", usdc_balance)

Mainnet funds to distribute
WETH: 79973292639517791570
WBTC: 16056508
TBTC: 49858198424970731
USDC: 39578598443


#### Mainnet user fund distribution before whitehat

> Block `t_0` is 1 block before the block where the first whitehat txn on a given chain executed
> Block `t_1` is the block of the first whitehat txn on a given chain

To calculate the funds owed to each user, we calculate the following at block `t_0` and `t_1` for each Panoptic user in each pool:
1. The net value of the user's options portfolio in token0 and token1, calculated by [`getNetLiquidationValue`](https://github.com/panoptic-labs/panoptic-v1-helper/blob/main/src/PanopticQuery.sol#L849) with arguments as follows:
    * pool - self explanatory
    * account - self explanatory
    * `includePendingPremium` - true
    * `positionIdList` - using open positions from subgraph at the block
    * `atTick` - using panoptic pool's TWAP tick at the block.
2. Collateral balance shares converted to assets, using [`convertToAssets`](https://github.com/panoptic-labs/panoptic-v1-core/blob/main/contracts/CollateralTracker.sol#L363).
    * There are two collateral trackers per pool, so these values also result in some amount of token0 and token1
* We net the token0 and token1 results from 1. and 2. to get the total assets in token0 and token1 each user had in the protocol at block `t_0`, right before the rescue, and `t_1`, right after the rescue.
* Subtract each user's token0 at `t_0` and token1 at `t_0`, from the token0 at `t_1` and token1 at `t_1` to get each user's loss in t0 and t1 from the rescue.

To get each user's total loss in each token, we net each user's loss in each token for each pool where they had deposits or open positions.

Finally, as a sanity check, we net all user losses for each token - this should be within a few percent of the respective token's balance held in our safe.

In [ ]:
t = mainnet_first_whitehat_block - 1

getNetLiquidationValue()

# what funds do we want to disburse?

# how to 